In [1]:
!pip install openai llama-index==0.10.27 llama-index-llms-openai==0.1.15
!pip install llama-index-embeddings-openai==0.1.7

  Using cached llama_index_core-0.10.68.post1-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_readers_file-0.1.33-py3-none-any.whl.metadata (5.4 kB)
Using cached llama_index_core-0.10.68.post1-py3-none-any.whl (1.6 MB)
Using cached llama_index_readers_file-0.1.33-py3-none-any.whl (38 kB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.11
    Uninstalling llama-index-core-0.11.11:
      Successfully uninstalled llama-index-core-0.11.11
  Attempting uninstall: llama-index-readers-file
    Found existing installation: llama-index-readers-file 0.2.2
    Uninstalling llama-index-readers-file-0.2.2:
      Successfully uninstalled llama-index-readers-file-0.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-fastembed 0.2.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama

In [2]:
import os
os.environ["OPENAI_API_KEY"] = 'open_ai_api'

In [3]:
import os
import openai
import nest_asyncio
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from typing import List, Optional
import subprocess
openai.api_key = os.getenv('OPENAI_API_KEY')

nest_asyncio.apply()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.vector_stores import MetadataFilters,FilterCondition
from llama_index.core.agent import AgentRunner
from llama_index.core.objects import ObjectIndex
from llama_index.core.tools import FunctionTool
from llama_index.llms.mistralai import MistralAI
from typing import List,Optional
import os
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings

In [5]:
!pip install fastembed llama-index-vector-stores-chroma llama-index-llms-mistralai llama-index-embeddings-fastembed llama-index-readers-file


  Using cached llama_index_core-0.11.11-py3-none-any.whl.metadata (2.4 kB)
INFO: pip is looking at multiple versions of llama-index-readers-file to determine which version is compatible with other requirements. This could take a while.
  Using cached llama_index_readers_file-0.2.2-py3-none-any.whl.metadata (5.4 kB)
Using cached llama_index_readers_file-0.2.2-py3-none-any.whl (38 kB)
Using cached llama_index_core-0.11.11-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
  Attempting uninstall: llama-index-readers-file
    Found existing installation: llama-index-readers-file 0.1.33
    Uninstalling llama-index-readers-file-0.1.33:
      Successfully uninstalled llama-index-readers-file-0.1.33
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [6]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]



def download_file(url, file_path):
    """Downloads a file from a given URL and saves it to the specified file path.

    Args:
        url: The URL of the file to download.
        file_path: The path to save the downloaded file.
    """
    import requests

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    with open(file_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # Filter out keep-alive new chunks
                f.write(chunk)

    print(f"Downloaded file from {url} to {file_path}")


for url, paper in zip(urls, papers):
    download_file(url, paper)


Downloaded file from https://openreview.net/pdf?id=VtmBAGCN7o to metagpt.pdf
Downloaded file from https://openreview.net/pdf?id=6PmJoRfdaK to longlora.pdf
Downloaded file from https://openreview.net/pdf?id=hSyW5go0v8 to selfrag.pdf


In [7]:

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("multidocument-agent")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [8]:
os.environ["MISTRAL_API_KEY"] = "ZGEyy3UMX5pjKTmd13J3jG8qBoxM6Lu6"
Settings.llm = MistralAI(model="mistral-large-latest")
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context
    )
    summary_index = SummaryIndex(nodes)

    def vector_query(
        query: str, page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over the MetaGPT paper.

        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts, condition=FilterCondition.OR
            ),
        )
        response = query_engine.query(query)
        return response

    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}", fn=vector_query
    )

    def summary_query(
        query: str,
    ) -> str:
        """Perform a summary of document
        query (str): the string query to be embedded.
        """
        summary_engine = summary_index.as_query_engine(
            response_mode="tree_summarize",
            use_async=True,
        )

        response = summary_engine.query(query)
        return response

    summary_tool = FunctionTool.from_defaults(
        fn=summary_query, name=f"summary_tool_{name}"
    )

    return vector_query_tool, summary_tool

In [10]:
!pip install pathlib
import pathlib

paper_to_tools_dict = {}
papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, pathlib.Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

Getting tools for paper: metagpt.pdf
Getting tools for paper: longlora.pdf
Getting tools for paper: selfrag.pdf


In [11]:
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [12]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)
tools = obj_retriever.retrieve("compare and contrast the papers self rag and metagpt")
#
print(tools[0].metadata)
print(tools[1].metadata)

ToolMetadata(description='vector_tool_metagpt(query: str, page_numbers: Optional[List[str]] = None) -> str\nUse to answer questions over the MetaGPT paper.\n\n        Useful if you have specific questions over the MetaGPT paper.\n        Always leave page_numbers as None UNLESS there is a specific page you want to search for.\n\n        Args:\n            query (str): the string query to be embedded.\n            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE\n                if we want to perform a vector search\n                over all pages. Otherwise, filter by the set of specified pages.\n\n        ', name='vector_tool_metagpt', fn_schema=<class 'pydantic.v1.main.vector_tool_metagpt'>, return_direct=False)
ToolMetadata(description='vector_tool_selfrag(query: str, page_numbers: Optional[List[str]] = None) -> str\nUse to answer questions over the MetaGPT paper.\n\n        Useful if you have specific questions over the MetaGPT paper.\n        Always leave 

In [13]:
# Create Agent Runner
vector_query_tool = paper_to_tools_dict['metagpt.pdf'][0]
summary_tool = paper_to_tools_dict['metagpt.pdf'][1]

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool], llm=Settings.llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [14]:
response = agent.query(
    "what are agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: what are agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "what are agent roles in MetaGPT, and then how they communicate with each other."}
=== Function Output ===
In MetaGPT, there are five defined agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. These agents communicate with each other using a shared message pool where they can publish structured messages and subscribe to relevant messages based on their profiles. This communication protocol allows agents to obtain directional information from other roles and public information from the environment.
=== LLM Response ===
In MetaGPT, there are five defined agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. These agents communicate with each other using a shared message pool where they can publish structured messages and subscribe to